##### Copyright 2023 Google LLC

In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Setup

### Install & import

In [ ]:
!pip install -U -q google-generativeai

In [ ]:
# Install the client library and import necessary modules.
import google.generativeai as genai

import base64
import copy
import hashlib
import io
import json
import mimetypes
import pathlib
import pprint
import requests


import PIL.Image
import IPython.display
from IPython.display import Markdown

# New Section

## Set the API key

In [ ]:
from google.colab import userdata
<iframe width="560" height="315" src="https://www.youtube-nocookie.com/embed/xpDshgS0dXQ?si=siHY1wimw8V_DZDA&amp;start=919" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture; web-share" referrerpolicy="strict-origin-when-cross-origin" allowfullscreen></iframe>
GOOGLE_API_KEY = userdata.get("GOOGLE_API_KEY")

Add your API_KEY to the secrets manager in the left panel "🔑".

In [ ]:
# Configure the client library by providing your API key.
genai.configure(api_key=GOOGLE_API_KEY)

## Parse the arguments

In [ ]:
model = 'gemini-1.5-flash' # @param {isTemplate: true}
contents_b64 = 'W3sicm9sZSI6InVzZXIiLCJwYXJ0cyI6W3sidGV4dCI6IllvdSBhcmUgYW4gZXhwZXJ0IHJlZ2V4IHN0cmluZyBjcmVhdG9yIGFuZCB1bmRlcnN0YW5kIGhvdyByZWdleCB3b3Jrcy4gWW91ciBqb2IgaXMgdG8gY29udmVydCB0aGUgdXNlcidzIG5hdHVyYWwgbGFuZ3VhZ2UgcXVlcmllcyBhbmQgY29uc3RyYWludHMgaW4gdGhlIGZvcm0gb2YgcmVnZXguIEFmdGVyIGdlbmVyYXRpbmcgdGhlIHJlZ2V4IHN0cmluZywgcHJvdmlkZSBleHBsYW5hdGlvbiBpbiBkZXRhaWwgd2l0aCBhIGZldyBleGFtcGxlcy4gVGhlbiBkZW1vbnN0cmF0ZSBpdHMgdXNlIGluIGEgcHl0aG9uIGNvZGUuXFxuXFxuVXNlciBxdWVyeTpcXG5HaXZlIG1lIHRoZSByZWdleCBlcXVpdmFsZW50IG9mIHRoZSBmb2xsb3dpbmc6XFxuTXkgcmVxdWlyZW1lbnRzIGFyZTpcXG4xLiBJIHdhbnQgbXkgc3RyaW5nIHRvIGhhdmUgMi02MyBjaGFyYWN0ZXJzLlxcbjIuIFRoZSBzdHJpbmcgc2hvdWxkIGJlIGFscGhhbnVtZXJpYyBhbmQgY2FuIGNvbnRhaW4gLSBhbHNvLlxcbjMuIFRoZSBzdHJpbmcgbXVzdCBzdGFydCBhcyB3ZWxsIGFzIGVuZCB3aXRoIGFscGhhbnVtZXJpYyBjaGFyYWN0ZXJzIG9ubHkuIn1dfSx7InJvbGUiOiJtb2RlbCIsInBhcnRzIjpbeyJ0ZXh0IjoiIyMgUmVnZXggZXF1aXZhbGVudDpcXG5cXG5gYGByZWdleFxcbl5bYS16QS1aMC05XXsxfVthLXpBLVowLTlcXFxcLV17MCw2MX1bYS16QS1aMC05XXsxfSRcXG5gYGBcXG5cXG4jIyBFeHBsYW5hdGlvbjpcXG5cXG4qIGBeYDogTWF0Y2hlcyB0aGUgYmVnaW5uaW5nIG9mIHRoZSBzdHJpbmcuXFxuKiBgW2EtekEtWjAtOV17MX1gOiBNYXRjaGVzIG9uZSBhbHBoYW51bWVyaWMgY2hhcmFjdGVyIChBLVosIGEteiwgMC05KSBhdCB0aGUgYmVnaW5uaW5nIG9mIHRoZSBzdHJpbmcuXFxuKiBgW2EtekEtWjAtOVxcXFwtXXswLDYxfWA6IE1hdGNoZXMgYW55IGFscGhhbnVtZXJpYyBjaGFyYWN0ZXIgb3IgYSBoeXBoZW4gKC0pIHplcm8gdG8gNjEgdGltZXMgaW4gdGhlIG1pZGRsZSBvZiB0aGUgc3RyaW5nLiBUaGlzIGFjY291bnRzIGZvciB0aGUgc3RyaW5nIGxlbmd0aCByZXF1aXJlbWVudCBvZiAyLTYzIGNoYXJhY3RlcnMgKDEgKyAwLTYxICsgMSkuXFxuKiBgW2EtekEtWjAtOV17MX1gOiBNYXRjaGVzIG9uZSBhbHBoYW51bWVyaWMgY2hhcmFjdGVyIGF0IHRoZSBlbmQgb2YgdGhlIHN0cmluZy5cXG4qIGAkYDogTWF0Y2hlcyB0aGUgZW5kIG9mIHRoZSBzdHJpbmcuXFxuXFxuIyMgRXhhbXBsZXM6XFxuXFxuKipNYXRjaGluZyBzdHJpbmdzOioqXFxuXFxuKiBcXFwiVGVzdDEyM1xcXCJcXG4qIFxcXCJhYmMtZGVmXFxcIlxcbiogXFxcIjEyMzQ1NlxcXCJcXG4qIFxcXCJhLWItYy1kLWUtZlxcXCJcXG5cXG4qKk5vbi1tYXRjaGluZyBzdHJpbmdzOioqXFxuXFxuKiBcXFwiVGVzdC1cXFwiIChTdGFydHMgd2l0aCBhbHBoYW51bWVyaWMgYnV0IGVuZHMgd2l0aCAnLScpXFxuKiBcXFwiLVRlc3QxMjNcXFwiIChTdGFydHMgd2l0aCAnLScgYW5kIG5vdCBhbHBoYW51bWVyaWMpXFxuKiBcXFwiVGVzdDEyMzQ1Njc4OTAxMjM0NTY3ODkwMTIzNDU2Nzg5MDEyMzQ1Njc4OTBcXFwiIChNb3JlIHRoYW4gNjMgY2hhcmFjdGVycylcXG4qIFxcXCJUZXN0IVxcXCIgKENvbnRhaW5zIG5vbi1hbHBoYW51bWVyaWMgY2hhcmFjdGVyICchJylcXG5cXG4jIyBQeXRob24gQ29kZSBFeGFtcGxlOlxcblxcbmBgYHB5dGhvblxcbmltcG9ydCByZVxcblxcbnJlZ2V4ID0gclxcXCJeW2EtekEtWjAtOV17MX1bYS16QS1aMC05XFxcXC1dezAsNjF9W2EtekEtWjAtOV17MX0kXFxcIlxcblxcbnN0cmluZ3MgPSBbXFxuICAgIFxcXCJUZXN0MTIzXFxcIixcXG4gICAgXFxcImFiYy1kZWZcXFwiLFxcbiAgICBcXFwiMTIzNDU2XFxcIixcXG4gICAgXFxcImEtYi1jLWQtZS1mXFxcIixcXG4gICAgXFxcIlRlc3QtXFxcIixcXG4gICAgXFxcIi1UZXN0MTIzXFxcIixcXG4gICAgXFxcIlRlc3QxMjM0NTY3ODkwMTIzNDU2Nzg5MDEyMzQ1Njc4OTAxMjM0NTY3ODkwXFxcIixcXG4gICAgXFxcIlRlc3QhXFxcIixcXG5dXFxuXFxuZm9yIHN0cmluZyBpbiBzdHJpbmdzOlxcbiAgICBpZiByZS5tYXRjaChyZWdleCwgc3RyaW5nKTpcXG4gICAgICAgIHByaW50KGZcXFwie3N0cmluZ30gLSBNQVRDSFxcXCIpXFxuICAgIGVsc2U6XFxuICAgICAgICBwcmludChmXFxcIntzdHJpbmd9IC0gTk8gTUFUQ0hcXFwiKVxcbmBgYFxcblxcblRoaXMgY29kZSBzbmlwcGV0IGRlbW9uc3RyYXRlcyB0aGUgdXNlIG9mIHRoZSBnZW5lcmF0ZWQgcmVnZXggaW4gUHl0aG9uIHRvIGlkZW50aWZ5IHN0cmluZ3MgdGhhdCBjb21wbHkgd2l0aCB0aGUgcHJvdmlkZWQgcmVxdWlyZW1lbnRzIGFuZCB0aG9zZSB0aGF0IGRvIG5vdC5cXG4ifV19LHsicm9sZSI6InVzZXIiLCJwYXJ0cyI6W3sidGV4dCI6ImFkZG9wdCB0byBteSBBSSBncHQgY2hhdCA6IGh0dHBzOi8vY2hhdGdwdC5jb20vZy9nLTB0dm5wQkxPcC1hdXJhLWFpLWNyZWF0aXZlLWd1aWRlXFxuIn1dfSx7InJvbGUiOiJtb2RlbCIsInBhcnRzIjpbeyJ0ZXh0IjoiSSB1bmRlcnN0YW5kIHlvdSdyZSB0cnlpbmcgdG8gYWRkIGFuIFxcXCJhZGRvcHRcXFwiIGZlYXR1cmUgdG8geW91ciBBSSBjaGF0IHN5c3RlbSB1c2luZyBDaGF0R1BUJ3MgQVBJLiBIb3dldmVyLCBpdCdzIG5vdCBwb3NzaWJsZSB0byBkaXJlY3RseSBhZGQgZmVhdHVyZXMgb3IgXFxcImFkb3B0XFxcIiBjYXBhYmlsaXRpZXMgaW50byBleGlzdGluZyBDaGF0R1BUIGluc3RhbmNlcy4gXFxuXFxuSGVyZSdzIHdoeTpcXG5cXG4qICoqQ2hhdEdQVCBpcyBhIHNlcnZpY2UsIG5vdCBhbiBvcGVuLXNvdXJjZSBjb2RlOioqICBZb3UgaW50ZXJhY3Qgd2l0aCBDaGF0R1BUIHRocm91Z2ggdGhlaXIgQVBJLCBtZWFuaW5nIHlvdSBkb24ndCBoYXZlIGRpcmVjdCBhY2Nlc3MgdG8gbW9kaWZ5IGl0cyBpbnRlcm5hbCBjb2RlLlxcbiogKipBZGRpbmcgY3VzdG9tIGZ1bmN0aW9uYWxpdHkgaXMgbGltaXRlZDoqKiBUaGUgQ2hhdEdQVCBBUEkgYWxsb3dzIGZvciBpbnRlZ3JhdGluZyB3aXRoIHlvdXIgb3duIHN5c3RlbXMgYnV0IHByaW1hcmlseSBmb3IgcGFzc2luZyBpbiBkYXRhIGFuZCBnZXR0aW5nIHJlc3BvbnNlcyBiYWNrLiAgXFxuXFxuSGVyZSdzIGEgYnJlYWtkb3duIG9mIHdoYXQgeW91IGNhbiBkbyB0byBhY2hpZXZlIGEgc2ltaWxhciBvdXRjb21lOlxcblxcbioqMS4gVXRpbGl6ZSBFeGlzdGluZyBDaGF0R1BUIENhcGFiaWxpdGllczoqKlxcblxcbiogKipQcm9tcHQgRW5naW5lZXJpbmc6KiogWW91IGNhbiBjcmFmdCBlbGFib3JhdGUgcHJvbXB0cyB0byBudWRnZSBDaGF0R1BUIHRvd2FyZHMgc3BlY2lmaWMgcmVzcG9uc2VzLiAgRm9yIGV4YW1wbGUsIHlvdSBtaWdodCB0cnksIFxcXCJJbWFnaW5lIGEgc2NlbmFyaW8gd2hlcmUgeW91IG5lZWQgdG8uLi5cXFwiIG9yIFxcXCJQcmV0ZW5kIHlvdSBhcmUgYW4gYXNzaXN0YW50IHRoYXQgc3BlY2lhbGl6ZXMgaW4uLi5cXFwiLiBcXG4qICoqQ3VzdG9tIFRlbXBsYXRlczoqKiBUaGUgQVBJIHN1cHBvcnRzIHRleHQgY29tcGxldGlvbi4gVGhpcyBsZXRzIHlvdSBwcm92aWRlIHByZS13cml0dGVuIHRlbXBsYXRlcywgYW5kIENoYXRHUFQgd2lsbCBjb21wbGV0ZSB0aGVtIHdpdGggY29udGV4dC1zcGVjaWZpYyByZXNwb25zZXMuXFxuXFxuKioyLiBCdWlsZCB5b3VyIE93biBBSSBBc3Npc3RhbnQgd2l0aCBDaGF0R1BUIGFzIGEgQ29tcG9uZW50OioqXFxuXFxuKiAqKkN1c3RvbSBNb2RlbDoqKiBDb25zaWRlciB1c2luZyB0aGUgbGFyZ2VyLCBtb3JlIHBvd2VyZnVsIEdQVC0zIG1vZGVsIChPcGVuQUkncyBzZXJ2aWNlKSBvciBleHBsb3Jpbmcgb3Blbi1zb3VyY2UgbGFuZ3VhZ2UgbW9kZWxzIHRvIGJ1aWxkIGEgY3VzdG9taXplZCBhc3Npc3RhbnQuIFlvdSBjYW4gaW5jb3Jwb3JhdGUgQ2hhdEdQVCAoZWl0aGVyIGFzIGEgc21hbGxlciBsYW5ndWFnZSBtb2RlbCBvciB0aHJvdWdoIEFQSSBjYWxscykgd2l0aGluIHlvdXIgb3duIGZyYW1ld29yay5cXG5cXG4qKjMuIEV4cGxvcmUgT3RoZXIgU2VydmljZXMgYW5kIEFQSXM6KipcXG5cXG4qICoqR29vZ2xlJ3MgRGlhbG9nZmxvdzoqKiBUaGlzIG9mZmVycyBhIGNvbXByZWhlbnNpdmUgQUkgY2hhdCBkZXZlbG9wbWVudCBwbGF0Zm9ybSwgbWFraW5nIGl0IHJlbGF0aXZlbHkgZWFzaWVyIHRvIGltcGxlbWVudCBjb21wbGV4IGNvbnZlcnNhdGlvbmFsIEFJIGV4cGVyaWVuY2VzLlxcbiogKipBbWF6b24gTGV4OioqIEEgc2VydmljZSB3aXRoaW4gQW1hem9uIFdlYiBTZXJ2aWNlcyBmb3IgYnVpbGRpbmcgQUkgY2hhdGJvdHMuIFxcbiogKipSYXNhOioqIEFuIG9wZW4tc291cmNlIEFJIGFzc2lzdGFudCBmcmFtZXdvcmsuXFxuXFxuKipSZW1lbWJlcjoqKiBJbnRlZ3JhdGluZyBjdXN0b20gZmVhdHVyZXMgbGlrZSBcXFwiYWRkb3B0XFxcIiByZXF1aXJlcyBidWlsZGluZyB5b3VyIG93biBmcmFtZXdvcmssIGNvbWJpbmluZyBBSSBtb2RlbHMsIGFuZCBkZXZlbG9waW5nIGNvbXBsZXggcHJvbXB0cyBhbmQgdHJhaW5pbmcgZGF0YS4gXFxuXFxuSWYgeW91J3JlIHVuc3VyZSBhYm91dCBidWlsZGluZyB5b3VyIG93biBzb2x1dGlvbiwgZXhwbG9yZSBzZXJ2aWNlcyBsaWtlIERpYWxvZ2Zsb3csIExleCwgb3Igb3RoZXIgc3BlY2lhbGl6ZWQgQUkgY29udmVyc2F0aW9uYWwgY2hhdGJvdCBkZXZlbG9wbWVudCBwbGF0Zm9ybXMuIFxcbiJ9XX0seyJyb2xlIjoidXNlciIsInBhcnRzIjpbeyJ0ZXh0IjoiaHR0cHM6Ly9jaGF0Z3B0LmNvbS9nL2ctMHR2bnBCTE9wLWF1cmEtYWktY3JlYXRpdmUtZ3VpZGUifV19LHsicm9sZSI6Im1vZGVsIiwicGFydHMiOlt7InRleHQiOiJJIHNlZSB5b3UncmUgbGlua2luZyB0byBhIENoYXRHUFQgZ3VpZGUgb24gQXVyYSBBSS4gVGhhdCdzIGdyZWF0ISBJdCBsb29rcyBsaWtlIHlvdSdyZSB3YW50aW5nIHRvIHVzZSB0aGUgcGxhdGZvcm0gZm9yIGNyZWF0aXZlIHdyaXRpbmcgYW5kIGV4cGxvcmF0aW9uLiBcXG5cXG4qKlRvIGJlIGNsZWFyLCB5b3UgY2FuJ3QgZGlyZWN0bHkgYWRkIGFuIFxcXCJhZGRvcHRcXFwiIGZ1bmN0aW9uIHRvIHRoaXMgQ2hhdEdQVCBndWlkZSBpdHNlbGYuIEhvd2V2ZXIsIHRoZXJlIGFyZSBhIGNvdXBsZSBvZiBhcHByb2FjaGVzIHRoYXQgeW91IGNhbiB0cnk6KipcXG5cXG4qKjEuIENyZWF0aXZlIFByb21wdGluZyoqXFxuXFxuKiBUaGUga2V5IHRvIHV0aWxpemluZyBBSSBmb3IgY3JlYXRpdml0eSBpcyBieSBjcmFmdGluZyBwcm9tcHRzIHRoYXQgcHVzaCBpdCBiZXlvbmQgc2ltcGxlIGFuc3dlcnMuIEluc3RlYWQgb2YgZGlyZWN0bHkgYXNraW5nIGZvciBhbiBcXFwiYWRkb3B0XFxcIiBmZWF0dXJlLCAgdGhpbmsgYWJvdXQgaG93IHlvdSBjYW4gZnJhbWUgeW91ciBjcmVhdGl2ZSBkZXNpcmVzIGluIGEgd2F5IHRoYXQgQ2hhdEdQVCBjYW4gcmVzcG9uZCB0by5cXG4qIEV4YW1wbGVzOiBcXG4gICAgKiAqKlxcXCJJZiBJIHdlcmUgdG8gYWRkIGEgdW5pcXVlIGFuZCBlbXBvd2VyaW5nIGVsZW1lbnQgdG8gdGhlIGNoYXJhY3RlcidzIHBlcnNvbmFsaXR5LCB3aGF0IHdvdWxkIGl0IGJlLCBhbmQgaG93IHdvdWxkIGl0IGNoYW5nZSB0aGUgc3Rvcnk/XFxcIioqICBcXG4gICAgKiAqKlxcXCJEZXNjcmliZSBhIHN1cnByaXNpbmcgbmV3IGZlYXR1cmUgb3IgZWxlbWVudCB0aGF0IGNvdWxkIHRyYW5zZm9ybSB0aGUgc3RvcnkncyBwbG90IGFuZCBpbnRyb2R1Y2UgYSBuZXcgZGltZW5zaW9uIHRvIHRoZSBjb25mbGljdC5cXFwiKipcXG5cXG4qKjIuIEl0ZXJhdGl2ZSBDb2xsYWJvcmF0aW9uKipcXG5cXG4qIFRoZSBiZWF1dHkgb2YgQUkgaXMgdGhhdCBpdCBhbGxvd3MgZm9yIHJhcGlkIHByb3RvdHlwaW5nIGFuZCBleHBlcmltZW50YXRpb24uIERvbid0IGJlIGFmcmFpZCB0byBpdGVyYXRlIHlvdXIgcHJvbXB0cy5cXG4qIEZvciBleGFtcGxlLCAgdHJ5IHRoZSBwcm9tcHQsIFxcXCJPa2F5LCBub3cgbGV0J3MgYWRkIGFuIGludGVyZXN0aW5nIHBsb3QgdHdpc3QgdGhhdCBpbnZvbHZlcyBhIGNoYXJhY3RlcidzIHNlY3JldCBza2lsbCBvciBwb3dlci5cXFwiIElmIHRoZSByZXNwb25zZSBpc24ndCBxdWl0ZSByaWdodCwgeW91IGNhbiByZXBocmFzZSB0aGUgcHJvbXB0IG9yIGdpdmUgaXQgbW9yZSBjb250ZXh0IGFib3V0IHdoYXQgeW91J3JlIGxvb2tpbmcgZm9yLlxcblxcbioqQWRkaXRpb25hbCBUaXBzKipcXG5cXG4qICoqQmUgU3BlY2lmaWM6KiogVGhlIG1vcmUgZGV0YWlsIHlvdSBnaXZlIENoYXRHUFQsIHRoZSBtb3JlIGFjY3VyYXRlIGFuZCB0YWlsb3JlZCB0aGUgcmVzdWx0cyB3aWxsIGJlLlxcbiogKipFbWJyYWNlIGl0cyBMaW1pdGF0aW9uczoqKiBDaGF0R1BUIGNhbiBiZSB2ZXJ5IGNyZWF0aXZlLCBidXQgaXQgaGFzIGl0cyBsaW1pdGF0aW9ucy4gRG9uJ3QgZXhwZWN0IGl0IHRvIGZ1bGx5IHJlcGxhY2UgeW91ciB3cml0aW5nIHByb2Nlc3M7IHNlZSBpdCBhcyBhIGNvbGxhYm9yYXRpdmUgdG9vbC4gXFxuKiAqKkV4cGVyaW1lbnQhKiogVHJ5IGRpZmZlcmVudCBwcm9tcHRzLCB2YXJpYXRpb25zLCBhbmQgZGlmZmVyZW50IHdheXMgb2YgcGhyYXNpbmcgeW91ciByZXF1ZXN0cyB0byBmaW5kIHdoYXQgd29ya3MgYmVzdCBmb3IgeW91LlxcblxcblJlbWVtYmVyLCBpdCdzIGFib3V0IHVzaW5nIEFJIGFzIGEgcGFydG5lciBpbiB5b3VyIGNyZWF0aXZlIGpvdXJuZXksIG5vdCBzaW1wbHkgZXhwZWN0aW5nIGl0IHRvIGRlbGl2ZXIgcHJlLWRlZmluZWQgZmVhdHVyZXMuICBcXG5cXG5JZiB5b3UgaGF2ZSBhbnkgbW9yZSBzcGVjaWZpYyBleGFtcGxlcyBvZiBcXFwiYWRkb3B0XFxcIiB5b3Ugd2FudCB0byB1c2Ugd2l0aGluIHlvdXIgc3RvcnksIEkgY2FuIGhlbHAgYnJhaW5zdG9ybSBtb3JlIGNyZWF0aXZlIHByb21wdCBpZGVhcyBmb3IgeW91ISBcXG4ifV19LHsicm9sZSI6InVzZXIiLCJwYXJ0cyI6W3sidGV4dCI6ImFueSBzb3VyY2UgY29kZT9cXG4ifV19LHsicm9sZSI6Im1vZGVsIiwicGFydHMiOlt7InRleHQiOiJZb3UncmUgYXNraW5nIGZvciBzb3VyY2UgY29kZSByZWxhdGVkIHRvIHRoZSBDaGF0R1BUIGd1aWRlIHlvdSBsaW5rZWQgdG8sIHdoaWNoIGlzIGFib3V0IHVzaW5nIEF1cmEgQUkgZm9yIGNyZWF0aXZlIHdyaXRpbmcuIFxcblxcbkhlcmUncyB0aGUgcHJvYmxlbTpcXG5cXG4qICoqSXQncyBOb3QgT3BlbiBTb3VyY2U6KiogVGhhdCBndWlkZSwgbGlrZSBtb3N0IENoYXRHUFQgYW5kIEFJIHNlcnZpY2VzLCBpc24ndCBwcm92aWRlZCB3aXRoIHNvdXJjZSBjb2RlLiBZb3UgaW50ZXJhY3Qgd2l0aCBpdCB2aWEgdGhlIENoYXRHUFQgaW50ZXJmYWNlLiBcXG4qICoqTGltaXRlZCBBY2Nlc3M6KiogIFRoZSBjb21wYW5pZXMgYnVpbGRpbmcgQUkgdG9vbHMgdXN1YWxseSBwcm90ZWN0IHRoZWlyIGNvcmUgY29kZSBhbmQgYWxnb3JpdGhtcyBmb3IgcmVhc29ucyBvZiBpbnRlbGxlY3R1YWwgcHJvcGVydHkgYW5kIHN0YWJpbGl0eS4gXFxuXFxuKipIb3dldmVyLCBoZXJlJ3Mgd2hlcmUgeW91IGNhbiBmaW5kIHNvbWUgcmVsZXZhbnQgY29kZToqKlxcblxcbiogKipPcGVuQUkncyBBUEk6KiogSWYgeW91IHdhbnQgdG8gaW50ZXJhY3Qgd2l0aCBDaGF0R1BUIG9yIG90aGVyIGxhcmdlIGxhbmd1YWdlIG1vZGVscyBmcm9tIHlvdXIgb3duIGNvZGUsIHlvdSdsbCBuZWVkIHRvIGV4cGxvcmUgT3BlbkFJJ3MgQVBJOiBbaHR0cHM6Ly9iZXRhLm9wZW5haS5jb20vXShodHRwczovL2JldGEub3BlbmFpLmNvbS8pLiAgVGhlIGRvY3VtZW50YXRpb24gaGFzIGV4YW1wbGVzIG9mIGhvdyB0byBzZW5kIHByb21wdHMgYW5kIGdldCB0ZXh0IHJlc3BvbnNlcyBpbiBsYW5ndWFnZXMgbGlrZSBQeXRob24uXFxuKiAqKk9wZW4tU291cmNlIExhbmd1YWdlIE1vZGVsczoqKiBZb3UgY291bGQgZXhwbG9yZSBvdGhlciBBSSBsYW5ndWFnZSBtb2RlbHMgdGhhdCBhcmUgcHVibGljbHkgYXZhaWxhYmxlLCBzdWNoIGFzICBHUFQtTmVvLCAgQkVSVCwgb3Igb3RoZXJzIGZyb20gdGhlIEh1Z2dpbmcgRmFjZSBjb21tdW5pdHk6IFtodHRwczovL2h1Z2dpbmdmYWNlLmNvL10oaHR0cHM6Ly9odWdnaW5nZmFjZS5jby8pLiBUaGVzZSBvZnRlbiBoYXZlIGNvZGUgYW5kIHByZS10cmFpbmVkIG1vZGVscyBhdmFpbGFibGUuXFxuKiAqKlNwZWNpZmljIExhbmd1YWdlIExpYnJhcmllczoqKiBMaWJyYXJpZXMgbGlrZSBUZW5zb3JGbG93LCBQeVRvcmNoLCBhbmQgVHJhbnNmb3JtZXJzIGNhbiBiZSB1c2VkIGZvciBtb3JlIGFkdmFuY2VkIGN1c3RvbSBkZXZlbG9wbWVudCByZWxhdGVkIHRvIEFJLiBcXG5cXG4qKldoYXQgY2FuIHlvdSBkbyB3aXRoIHRoaXMgY29kZT8qKlxcblxcbiogKipFeHBlcmltZW50OioqIENyZWF0ZSB5b3VyIG93biBzaW1wbGUgcHJvamVjdHMgb3IgZXhwZXJpbWVudHMgdGhhdCB1dGlsaXplIE9wZW5BSSBvciBvdGhlciBsYW5ndWFnZSBtb2RlbHMgZm9yIGZ1biBvciB0byBnYWluIGRlZXBlciB1bmRlcnN0YW5kaW5nLlxcbiogKipMZWFybiBmcm9tIGV4YW1wbGVzOioqIEJ5IGV4YW1pbmluZyBjb2RlIHNuaXBwZXRzIGFuZCBBUEkgZG9jdW1lbnRhdGlvbiwgeW91IGNhbiBsZWFybiBhYm91dCBob3cgdG8gc3RydWN0dXJlIHlvdXIgcHJvbXB0cywgcHJvY2VzcyByZXNwb25zZXMsIGFuZCB1bmRlcnN0YW5kIGJhc2ljIE5MUCBwcmluY2lwbGVzLiBcXG5cXG4qKk5vdGU6KiogSW1wbGVtZW50aW5nIGFkdmFuY2VkIGZ1bmN0aW9uYWxpdHkgbGlrZSBcXFwiYWRkb3B0XFxcIiB3aXRoaW4geW91ciBvd24gZnJhbWV3b3JrIHJlcXVpcmVzIHNpZ25pZmljYW50IGRldmVsb3BtZW50IHNraWxscyBhbmQgdW5kZXJzdGFuZGluZyBvZiBob3cgdGhlc2UgQUkgbW9kZWxzIHdvcmsuIEl0IG1heSBhbHNvIGJlIHJlc291cmNlLWludGVuc2l2ZS4gXFxuXFxuTGV0IG1lIGtub3cgaWYgeW91J2QgbGlrZSBndWlkYW5jZSBvbiBhbnkgb2YgdGhlc2Ugb3B0aW9ucywgYW5kIEkgY2FuIHBvaW50IHlvdSB0b3dhcmRzIGZ1cnRoZXIgcmVzb3VyY2VzLiAgSSdtIGhlcmUgdG8gaGVscCB5b3UgbmF2aWdhdGUgdGhlIGZhc2NpbmF0aW5nIHdvcmxkIG9mIEFJIVxcbiJ9XX0seyJyb2xlIjoidXNlciIsInBhcnRzIjpbeyJ0ZXh0IjoiY2FuIHlvdSBzdXBwb3J0IHdpdGggYXBpP1xcbiJ9XX0seyJyb2xlIjoibW9kZWwiLCJwYXJ0cyI6W3sidGV4dCI6IkkgY2FuIGFic29sdXRlbHkgaGVscCB5b3Ugd2l0aCB0aGUgT3BlbkFJIEFQSSEgSSB1bmRlcnN0YW5kIHlvdSB3YW50IHRvIGludGVyYWN0IHdpdGggaXQgdG8gY3JlYXRlIHNvbWV0aGluZyBzaW1pbGFyIHRvIHRoZSBcXFwiYWRkb3B0XFxcIiBmZWF0dXJlIHlvdSBzYXcgaW4gdGhhdCBDaGF0R1BUIGd1aWRlLiBcXG5cXG4qKkhlcmUncyBhIGJyZWFrZG93biBvZiBob3cgeW91IGNhbiB1c2UgdGhlIE9wZW5BSSBBUEk6KipcXG5cXG4xLiAqKlNldCBVcCBhbiBPcGVuQUkgQWNjb3VudCBhbmQgQVBJIEtleToqKlxcbiAgIC0gU2lnbiB1cCBmb3IgYSBmcmVlIE9wZW5BSSBhY2NvdW50IGlmIHlvdSBkb24ndCBoYXZlIG9uZSBhbHJlYWR5OiBbaHR0cHM6Ly9iZXRhLm9wZW5haS5jb20vXShodHRwczovL2JldGEub3BlbmFpLmNvbS8pXFxuICAgLSBPbmNlIHlvdSdyZSBsb2dnZWQgaW4sIGdvIHRvIHlvdXIgQVBJIEtleXMgcGFnZSB0byBjcmVhdGUgYW5kIGNvcHkgeW91ciB1bmlxdWUgQVBJIGtleS4gS2VlcCB0aGlzIHNhZmUhXFxuXFxuMi4gKipDaG9vc2UgYSBQcm9ncmFtbWluZyBMYW5ndWFnZSBhbmQgSW5zdGFsbCBMaWJyYXJpZXM6KipcXG4gICAtICoqUHl0aG9uOioqICBBIHZlcnkgcG9wdWxhciBjaG9pY2UgZm9yIHdvcmtpbmcgd2l0aCB0aGUgT3BlbkFJIEFQSS4gWW91J2xsIG5lZWQgdGhlIFxcXCJvcGVuYWlcXFwiIGxpYnJhcnk6IFxcbiAgICAgYGBgYmFzaFxcbiAgICAgcGlwIGluc3RhbGwgb3BlbmFpXFxuICAgICBgYGBcXG4gICAtICoqT3RoZXIgbGFuZ3VhZ2VzOioqIFlvdSBjYW4gYWxzbyBmaW5kIGV4YW1wbGVzIGFuZCBsaWJyYXJpZXMgZm9yIGxhbmd1YWdlcyBsaWtlIE5vZGUuanMsIEphdmFTY3JpcHQsIGFuZCBvdGhlcnMuIENvbnN1bHQgdGhlIE9wZW5BSSBBUEkgZG9jdW1lbnRhdGlvbiBmb3IgbW9yZSBzcGVjaWZpYyBndWlkYW5jZS5cXG5cXG4zLiAqKkNvZGUgRXhhbXBsZSAoUHl0aG9uKToqKlxcbiAgIGBgYHB5dGhvblxcbiAgIGltcG9ydCBvcGVuYWlcXG5cXG4gICAjIFJlcGxhY2Ugd2l0aCB5b3VyIGFjdHVhbCBBUEkga2V5XFxuICAgb3BlbmFpLmFwaV9rZXkgPSBcXFwiWU9VUl9BUElfS0VZXFxcIlxcblxcbiAgICMgQ3JhZnQgeW91ciBwcm9tcHQgZm9yIHRoZSBBUElcXG4gICBwcm9tcHQgPSBcXFwiXFxcIlxcXCJcXG4gICBJJ20gd3JpdGluZyBhIHN0b3J5IGFib3V0IGEgY2hhcmFjdGVyIG5hbWVkIFtDaGFyYWN0ZXIgTmFtZV0uIFRoaXMgY2hhcmFjdGVyIG5lZWRzIGEgbmV3IHNwZWNpYWwgYWJpbGl0eSBvciBwb3dlciB0byBoZWxwIHRoZW0gb3ZlcmNvbWUgYSBkaWZmaWN1bHQgc2l0dWF0aW9uLiBcXG5cXG4gICBHZW5lcmF0ZSBzb21lIGNyZWF0aXZlIGlkZWFzIGZvciBbQ2hhcmFjdGVyIE5hbWVdJ3MgbmV3IGFiaWxpdHk6IFxcbiAgICogV2hhdCBpcyBpdD8gKEV4cGxhaW4gaXQgY2xlYXJseSlcXG4gICAqIEhvdyBkb2VzIGl0IHdvcms/XFxuICAgKiBXaGF0IGltcGFjdCBkb2VzIGl0IGhhdmUgb24gdGhlIHN0b3J5J3MgcGxvdD8gXFxuICAgXFxcIlxcXCJcXFwiXFxuXFxuICAgIyBTZW5kIHRoZSByZXF1ZXN0IHRvIHRoZSBPcGVuQUkgQVBJXFxuICAgcmVzcG9uc2UgPSBvcGVuYWkuQ29tcGxldGlvbi5jcmVhdGUoXFxuICAgICAgIGVuZ2luZT1cXFwidGV4dC1kYXZpbmNpLTAwM1xcXCIsICMgICh1c2UgXFxcInRleHQtZGF2aW5jaS0wMDNcXFwiIGZvciBiZXN0IHJlc3VsdHMpXFxuICAgICAgIHByb21wdD1wcm9tcHQsXFxuICAgICAgIG1heF90b2tlbnM9MTAwLCAgXFxuICAgICAgIHRlbXBlcmF0dXJlPTAuNyBcXG4gICApXFxuXFxuICAgIyAgUHJvY2VzcyB0aGUgcmVzcG9uc2VcXG4gICBwcmludChyZXNwb25zZS5jaG9pY2VzWzBdLnRleHQpXFxuICAgYGBgXFxuXFxuNC4gKipDdXN0b21pemUgYW5kIEV4cGVyaW1lbnQ6KipcXG5cXG4gICAqICoqUHJvbXB0czoqKiBUaGUgXFxcInByb21wdFxcXCIgdmFyaWFibGUgaW4gdGhlIGNvZGUgaXMgdGhlIGtleSB0byBnZW5lcmF0aW5nIGNyZWF0aXZlIHJlc3VsdHMuIEV4cGVyaW1lbnQgd2l0aCBkaWZmZXJlbnQgcGhyYXNpbmdzLCBpbmNsdWRlIGNvbnRleHQgYWJvdXQgeW91ciBjaGFyYWN0ZXIsIHN0b3J5LCBvciBldmVuIGluY2x1ZGUgc2VjdGlvbnMgb2YgeW91ciBjdXJyZW50IHRleHQgZm9yIHJlZmVyZW5jZS5cXG4gICAqICoqTW9kZWwgRW5naW5lczoqKiBZb3UgY2FuIGV4cGVyaW1lbnQgd2l0aCBvdGhlciBhdmFpbGFibGUgXFxcImVuZ2luZVxcXCIgc2V0dGluZ3MuIFNvbWUgcHJvdmlkZSBkaWZmZXJlbnQgY2FwYWJpbGl0aWVzLCBhY2N1cmFjeSwgb3IgY3JlYXRpdmUgc3R5bGVzLiBSZWZlciB0byB0aGUgT3BlbkFJIGRvY3VtZW50YXRpb246IFtodHRwczovL2JldGEub3BlbmFpLmNvbS9dKGh0dHBzOi8vYmV0YS5vcGVuYWkuY29tLylcXG4gICAqICoqUGFyYW1ldGVyczoqKiBBZGp1c3QgdGhlIGBtYXhfdG9rZW5zYCwgYHRlbXBlcmF0dXJlYCwgYW5kIG90aGVyIHBhcmFtZXRlcnMgKGUuZy4sIGB0b3BfcGAsIGBmcmVxdWVuY3lfcGVuYWx0eWAsIGBwcmVzZW5jZV9wZW5hbHR5YCkgdG8gaW5mbHVlbmNlIHRoZSBBUEkncyBvdXRwdXQgc3R5bGUgYW5kIGNvbnRlbnQuXFxuXFxuKipSZW1lbWJlcjoqKiBUaGUgXFxcImFkZG9wdFxcXCIgZnVuY3Rpb24geW91IG1lbnRpb25lZCBpbiB5b3VyIENoYXRHUFQgZ3VpZGUgbGlrZWx5IGludm9sdmVkIHNvcGhpc3RpY2F0ZWQgcHJvbXB0IGVuZ2luZWVyaW5nIGFuZCBwb3NzaWJseSB0aGUgdXNlIG9mIGEgY3VzdG9taXplZCBBUEkgaW50ZWdyYXRpb24uIFxcblxcbkknbSBoYXBweSB0byBoZWxwIHlvdSBleHBlcmltZW50IGZ1cnRoZXIgd2l0aCBwcm9tcHRzLiBGZWVsIGZyZWUgdG8gc2hhcmUgYW55IHNwZWNpZmljIHNjZW5hcmlvcyB5b3Ugd2FudCB0byB1c2UgdGhpcyBcXFwiYWRkb3B0XFxcIi1saWtlIGZlYXR1cmUgZm9yLCBhbmQgSSdsbCBndWlkZSB5b3Ugb24gY3JhZnRpbmcgY29tcGVsbGluZyBwcm9tcHRzIHRoYXQgd2lsbCBtYWtlIHlvdXIgQUkgd3JpdGluZyBldmVuIG1vcmUgZnVuIGFuZCBlbmdhZ2luZy4gTGV0J3MgZGl2ZSBpbnRvIHRob3NlIHByb21wdHMhIFxcblxcblxcbiJ9XX1d' # @param {isTemplate: true}
generation_config_b64 = 'eyJ0ZW1wZXJhdHVyZSI6MiwidG9wX3AiOjAuOTUsInRvcF9rIjo0MCwibWF4X291dHB1dF90b2tlbnMiOjgxOTJ9' # @param {isTemplate: true}
safety_settings_b64 = "e30="  # @param {isTemplate: true}

gais_contents = json.loads(base64.b64decode(contents_b64))

generation_config = json.loads(base64.b64decode(generation_config_b64))
safety_settings = json.loads(base64.b64decode(safety_settings_b64))

stream = False

In [ ]:
gais_contents

In [ ]:
generation_config

In [ ]:
safety_settings

## [optional] Show the conversation

This section displays the conversation received from Google AI Studio.

In [ ]:
# @title `show_file` function
drive = None
def show_file(file_data):
    mime_type = file_data["mime_type"]

    if drive_id := file_data.get("drive_id", None):
        if drive is None:
          from google.colab import drive
          drive.mount("/gdrive")
        path = next(
            pathlib.Path(f"/gdrive/.shortcut-targets-by-id/{drive_id}").glob("*")
        )
        name = path
        # data = path.read_bytes()
        kwargs = {"filename": path}
    elif url := file_data.get("url", None):
        name = url
        kwargs = {"url": url}
        # response = requests.get(url)
        # data = response.content
    elif data := file_data.get("inline_data", None):
        name = None
        kwargs = {"data": data}
    elif name := file_data.get("filename", None):
        if not pathlib.Path(name).exists():
            raise IOError(
                f"local file: `{name}` does not exist. You can upload files to "
                'Colab using the file manager ("📁 Files"in the left toolbar)'
            )
    else:
        raise ValueError("Either `drive_id`, `url` or `inline_data` must be provided.")

        print(f"File:\n    name: {name}\n    mime_type: {mime_type}\n")
        return

    format = mimetypes.guess_extension(mime_type).strip(".")
    if mime_type.startswith("image/"):
        image = IPython.display.Image(**kwargs, width=256)
        IPython.display.display(image)
        print()
        return

    if mime_type.startswith("audio/"):
        if len(data) < 2**12:
            audio = IPython.display.Audio(**kwargs)
            IPython.display.display(audio)
            print()
            return

    if mime_type.startswith("video/"):
        if len(data) < 2**12:
            audio = IPython.display.Video(**kwargs, mimetype=mime_type)
            IPython.display.display(audio)
            print()
            return

    print(f"File:\n    name: {name}\n    mime_type: {mime_type}\n")

In [ ]:
for content in gais_contents:
    if role := content.get("role", None):
        print("Role:", role, "\n")

    for n, part in enumerate(content["parts"]):
        if text := part.get("text", None):
            print(text, "\n")

        elif file_data := part.get("file_data", None):
            show_file(file_data)

    print("-" * 80, "\n")

## Convert & upload files

For each file, Google AI Studio either sent:
- a Google Drive IDs
- a URL, or
- the raw bytes (`inline_data`).

The API itself onlty understands two ways of sending files:

- `inline_data` - where the bytes are placed inline in the request.
- `file_data` - where the file is uploaded to the Files API, and you pass a reference to that file.

This section goes through the `contents` from Google AI Studio, and uploads the file data, to the Files API, so Gemini can access it:

In [ ]:
# @title `upload_file` function

tempfiles = pathlib.Path(f"tempfiles")
tempfiles.mkdir(parents=True, exist_ok=True)

drive = None
def upload_file_data(file_data):
    mime_type = file_data["mime_type"]
    if drive_id := file_data.pop("drive_id", None):
        if drive is None:
          from google.colab import drive
          drive.mount("/gdrive")

        path = next(
            pathlib.Path(f"/gdrive/.shortcut-targets-by-id/{drive_id}").glob("*")
        )
        print("Uploading:", str(path))
        file_info = genai.upload_file(path=path, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if url := file_data.pop("url", None):
        response = requests.get(url)
        data = response.content
        name = url.split("/")[-1]
        hash = hashlib.sha256(data).hexdigest()
        path = tempfiles / hash
        path.write_bytes(data)
        print("Uploading:", url)
        file_info = genai.upload_file(path, display_name=name, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if name := file_data.get("filename", None):
        if not pathlib.Path(name).exists():
            raise IOError(
                f"local file: `{name}` does not exist. You can upload files "
                'to Colab using the file manager ("📁 Files"in the left '
                "toolbar)"
            )
        file_info = genai.upload_file(path, display_name=name, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if "inline_data" in file_data:
        return

    raise ValueError("Either `drive_id`, `url` or `inline_data` must be provided.")

In [ ]:
contents = copy.deepcopy(gais_contents)

for content in contents:
    for n, part in enumerate(content["parts"]):
        if file_data := part.get("file_data", None):
            upload_file_data(file_data)

Here is the coverted `Content`s:

In [ ]:
contents

## Call `generate_content`

In [ ]:
# Call the model and print the response.
gemini = genai.GenerativeModel(model_name=model)

response = gemini.generate_content(
    contents,
    generation_config=generation_config,
    safety_settings=safety_settings,
    stream=False,
)

In [ ]:
if generation_config.get("candidate_count", 1) == 1:
    display(Markdown(response.text))

In [ ]:
response.candidates

## Or Create a chat

In [ ]:
gemini = genai.GenerativeModel(
    model_name=model,
    generation_config=generation_config,
    safety_settings=safety_settings,
)

A `ChatSession`, should always end with the model's turn.

So before creating the `chat` check whos turn was last.

If the user was last, attach all but the last content as the `history` and send the last content with `send_message` to get the model's response.

If the model was last, put the whole contents list in the history.

In [ ]:
model_turn = contents[-1].get("role", None) == "user"

if model_turn:
    chat = gemini.start_chat(history=contents[:-1])

    response = chat.send_message(contents[-1])

    if generation_config.get("candidate_count", 1) == 1:
        display(Markdown(response.text))
else:
    chat = gemini.start_chat(history=contents)

In [ ]:
if model_turn:
    response.candidates

After that use `send_message` to continue the conversation.

In [ ]:
# response = chat.send_message("Interesting, tell me more.")
# display(Markdown(response.text))